In [85]:
from ftplib import FTP

from astropy.time import Time, TimeDelta
from astropy import units as u
# для astropy в режиме offline
from astropy.utils import iers
iers.conf.auto_download = False  

from secret_key import login, password

In [86]:
"""
N1 – 194.149.67.97 (от 20° до 42°)
N2 – 194.149.67.38 (от -9° до 20°)
"""

con = FTP('194.149.67.97', login, password)

In [87]:
data = []
con.dir(data.append)

In [88]:
file_list = con.nlst('*.pnt')

In [89]:
data = []
def read_pnt_header(line):
    data.append(line.decode("utf-8"))

In [90]:
def get_time_begin_and_end(header):
    if 'UTC' in header['date_begin']:
        # begin
        day, month, year = header['date_begin'][2].split('.')
        hour, minute, second = header['time_begin'][2].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_start = Time(isot_time, format='isot', scale='utc', precision=7)
        # end
        day, month, year = header['date_end'][0].split('.')
        hour, minute, second = header['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_end = Time(isot_time, format='isot', scale='utc', precision=7)
    else:
        # begin
        day, month, year = header['date_begin'][0].split('.')
        hour, minute, second = header['time_begin'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_start = Time(isot_time, format='isot', scale='utc', precision=7)
        time_start -= 4*u.hour
        # end
        day, month, year = header['date_end'][0].split('.')
        hour, minute, second = header['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_end = Time(isot_time, format='isot', scale='utc', precision=7)
        time_end -= 4*u.hour
        
    return time_start, time_end
    

In [91]:
try:
    con.retrbinary('RETR ' + file_list[1], callback=read_pnt_header, blocksize=493)
except UnicodeDecodeError:
    print('Read was done!')
except ConnectionResetError:
    print('!!! Connection was aborted !!!')

Read was done!


In [92]:
header = {}
for line in data[0].split('\n'):
    a, *b = line.split()
    header[a] = b

In [93]:
header

{'alpha': ['alpha'],
 'date_begin': ['01.04.2019', 'UTC', '31.03.2019'],
 'date_end': ['01.04.2019'],
 'delta': ['delta'],
 'fbands': ['109.20751953125',
  '109.62255859375',
  '110.03759765625',
  '110.45263671875',
  '110.86767578125',
  '111.28759765625'],
 'fcentral': ['110.25'],
 'modulus': ['1', '2', '3', '4', '5', '6'],
 'nbands': ['6'],
 'npoints': ['36018'],
 'numpar': ['16'],
 'source': ['source'],
 'time_begin': ['0:00:00', 'UTC', '20:00:00'],
 'time_end': ['0:59:59'],
 'tresolution': ['99.9424'],
 'wb_total': ['2.5'],
 'wbands': ['0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4248046875']}

In [94]:
t_b, t_e = get_time_begin_and_end(header)

In [95]:
print(t_b, t_e)

2019-03-31T20:00:00.0000000 2019-04-01T00:59:59.0000000
